# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Импортируем нужные библиотеки, посмотрим общие данные, проверим на дубликаты.

In [1]:
import pandas as pd# Импортирую нужные в работе библиотеки
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score 

In [2]:
try:

    df_1= pd.read_csv('C:/Users/Пользователь/OneDrive/Документы/Python/Churn.csv')
   
except:

    df = pd.read_csv('/datasets/Churn.csv')# Откроем фаил

In [3]:
df.info()# Посмотрим общую информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
df.head(10)# взглянем на первые 10 строчек 

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [5]:
df.columns = map(str.lower, df.columns)# преведем названия столбцов к нижнему регистру

In [6]:
df.isna().sum()# посмотрим количество пропущенных значений для каждого столбца

rownumber            0
customerid           0
surname              0
creditscore          0
geography            0
gender               0
age                  0
tenure             909
balance              0
numofproducts        0
hascrcard            0
isactivemember       0
estimatedsalary      0
exited               0
dtype: int64

In [7]:
df['tenure'].unique()# изучем оригинальные значения столбца с пропусками

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

In [8]:
df['tenure'].value_counts()# Посмотрим распределение по значениям в столбце

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

In [9]:
df['tenure'].median()# посчитаем медиану

5.0

In [10]:
df['tenure'] = df['tenure'].fillna(5.0)# Пропуски заполним медианой

In [11]:
df['tenure'] = df['tenure'].astype(int)# поменяем тип данных

In [12]:
duplicate_df = df[df.duplicated()]
duplicate_df# проверим наши исходные данные на явные дубликаты

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited


In [13]:
df['customerid'].value_counts()# проверим наши исходные данные на неявные дубликаты

15695872    1
15801062    1
15682268    1
15647453    1
15684319    1
           ..
15629677    1
15773039    1
15766896    1
15719793    1
15812607    1
Name: customerid, Length: 10000, dtype: int64

## Исследование задачи

Итак для выявления наиболее подходящей модели сначала надо сравнить показатели F1 между собой. Приступим!

In [14]:
df_ohe = df.drop(['rownumber', 'customerid', 'surname'], axis=1)# удалим ненужные для модели столбцы

In [15]:
df_ohe = pd.get_dummies(df_ohe, drop_first=True)# применим прямое кодирование

In [16]:
target = df_ohe['exited']
features = df_ohe.drop(['exited'], axis=1)

In [17]:
features_tr, features_valid, target_tr, target_valid = train_test_split(features, target, test_size=0.2, stratify=df_ohe['exited'], random_state=12345)
# поделим данные на выборки

In [18]:
features_train, features_test, target_train, target_test = train_test_split(features_tr, target_tr, test_size=0.25, random_state=12345)
# поделим данные на выборки

In [19]:
numeric = ['creditscore', 'age', 'balance', 'estimatedsalary', 'tenure']# Масштабируем данные

In [20]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])# Масштабируем данные

StandardScaler()

In [21]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])# Масштабируем данные

/tmp/ipykernel_260/3330657556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric] = scaler.transform(features_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/tmp/ipykernel_260/3330657556.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Предварительная работа окончена. Пора строить модели!

Начнем с LogisticRegression.

In [22]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train) 
predicted = model.predict(features_valid)
f1_score(target_valid, predicted)

0.29158878504672897

In [23]:
probabilities = model.predict_proba(features_valid)
probabilities_one_valid = probabilities[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.775539792488945

Ого. результат ниже 0.3. Это никуда не годится)

Продолжим с DecisionTreeClassifier.

In [24]:
best_model_2= None
best_result_2= 0

for depth in range(1, 23):
    model_2 = DecisionTreeClassifier(random_state=12345, max_depth=depth)# применим метод
    model_2.fit(features_train, target_train)# обучим модель
    predicted_2 = model_2.predict(features_valid)
    result_2 = f1_score(target_valid, predicted_2)# качество модели посчитаем на валидационной выборке
    if result_2 > best_result_2:
        best_model_2= model_2# сохраним наилучшую модель
        best_result_2= result_2#  сохраним наилучшее значение метрики F1 на валидационных данных
        best_depth = depth

best_result_2, best_depth

(0.5710059171597632, 7)

In [25]:
probabilities_2= model_2.predict_proba(features_valid)
probabilities_one_valid_2 = probabilities_2[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid_2)
auc_roc

0.6709567811262726

0.57 уже заметно лучше.

И не забудем про RandomForestClassifier.

In [26]:
best_model_3 = None
best_result_3 = 0


for est in range(1, 20):
    for depth in range (1, 20):
        model_3 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_3.fit(features_train, target_train)
        predicted_3 = model_3.predict(features_valid)
        result_3 = f1_score(target_valid, predicted_3)
        
        if result_3 > best_result_3:
            best_model_3 = model_3
            best_result_3 = result_3
            best_est_3 = est
            best_depth_3 = depth
            
best_result_3, best_depth_3, best_est_3

(0.5844748858447488, 14, 14)

In [27]:
probabilities_3= model_3.predict_proba(features_valid)
probabilities_one_valid_3 = probabilities_3[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid_3)
auc_roc

0.8369918454664216

Результат почти не отличается от DecisionTreeClassifier, но все же немного выше.

До желаемых 0.59 не добрались. Будем бороться с дисбалансом!

## Борьба с дисбалансом

Сперва посмотрим как повлияет на результаты F1 взвешивание классов.

Порядок изучения моделей менять не будем.

In [28]:
model_4 = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_4.fit(features_train, target_train) 
predicted_valid = model_4.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.5068027210884353

In [29]:
probabilities_4= model_4.predict_proba(features_valid)
probabilities_one_valid_4 = probabilities_4[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid_4)
auc_roc

0.7815087815087813

In [30]:
best_model_5= None
best_result_5= 0

for depth in range(1, 50):
    model_5 = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced')# применим метод
    model_5.fit(features_train, target_train)# обучим модель
    predicted_5 = model_5.predict(features_valid)
    result_5 = f1_score(target_valid, predicted_5)# качество модели посчитаем на валидационной выборке
    if result_5 > best_result_5:
        best_model_5= model_5# сохраним наилучшую модель
        best_result_5= result_5#  сохраним наилучшее значение метрики F1 на валидационных данных
        best_depth = depth

best_result_5, best_depth

(0.5719733079122974, 6)

In [31]:
probabilities_5= model_5.predict_proba(features_valid)
probabilities_one_valid_5 = probabilities_5[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid_5)
auc_roc

0.6934353459777189

In [32]:
best_model_6 = None
best_result_6 = 0


for est in range(1, 25):
    for depth in range (1, 20):
        model_6 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced')
        model_6.fit(features_train, target_train)
        predicted_6 = model_6.predict(features_valid)
        result_6 = f1_score(target_valid, predicted_6)
        
        if result_6 > best_result_6:
            best_model_6 = model_6
            best_result_6 = result_6
            best_est_6 = est
            best_depth_6 = depth
            
best_result_6, best_depth_6, best_est_6

(0.6470588235294117, 9, 21)

In [33]:
probabilities_6= model_6.predict_proba(features_valid)
probabilities_one_valid_6 = probabilities_6[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid_6)
auc_roc

0.8460077951603376

Ну наконец то! Рубеж в 0.59 преодолен! при глубине в 7 и колличестве деревьев 23 метрика F1 показывает 0.62!

Неплохо, но лучше проверить и результаты с увеличением выборки.

In [34]:
features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]

In [35]:
repeat = 4
features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

In [36]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled



In [37]:
model_7 = LogisticRegression(random_state=12345, solver='liblinear')
model_7.fit(features_upsampled, target_upsampled)
predicted= model_7.predict(features_valid)
f1_score(target_valid, predicted)

0.5055131467345207

In [38]:
probabilities_7= model_7.predict_proba(features_valid)
probabilities_one_valid_7 = probabilities_7[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid_7)
auc_roc

0.7815072391343577

Модель LogisticRegression в любом варианте выдает худший показатель.

In [39]:
best_model_8= None
best_result_8= 0

for depth in range(1, 50):
    model_8 = DecisionTreeClassifier(random_state=12345, max_depth=depth)# применим метод
    model_8.fit(features_upsampled, target_upsampled)# обучим модель
    predicted_8 = model_8.predict(features_valid)
    result_8 = f1_score(target_valid, predicted_8)# качество модели посчитаем на валидационной выборке
    if result_8 > best_result_8:
        best_model_8= model_8# сохраним наилучшую модель
        best_result_8= result_8#  сохраним наилучшее значение метрики F1 на валидационных данных
        best_depth = depth

best_result_8, best_depth

(0.5719733079122974, 6)

In [40]:
probabilities_8= model_8.predict_proba(features_valid)
probabilities_one_valid_8 = probabilities_8[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid_8)
auc_roc

0.7054604681723325

In [41]:
best_model_9 = None
best_result_9 = 0


for est in range(1, 20):
    for depth in range (1, 20):
        model_9 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_9.fit(features_upsampled, target_upsampled)
        predicted_9 = model_6.predict(features_valid)
        result_9 = f1_score(target_valid, predicted_9)
        
        if result_9 > best_result_9:
            best_model_9 = model_9
            best_result_9 = result_9
            best_est_9 = est
            best_depth_9 = depth
            
best_result_9, best_depth_9, best_est_9

(0.5366614664586584, 1, 1)

In [42]:
probabilities_9= model_9.predict_proba(features_valid)
probabilities_one_valid_9 = probabilities_9[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid_9)
auc_roc

0.8466733297241771

На моделях DecisionTreeClassifier и RandomForestClassifier показатели так же хуже, чем в моделях со взвешиванием.

Подведем итог. Наивысший показатель F1 у модели RandomForestClassifier со взвешиванием классов!

Перейдем к тестированию.

## Тестирование модели

In [43]:
model_10 = RandomForestClassifier(random_state=12345, n_estimators=19, max_depth=7, class_weight='balanced')
model_10.fit(features_train, target_train)
predicted_10 = model_10.predict(features_test)
result_10 = f1_score(target_test, predicted_10)
result_10    

0.6148936170212767

После проверки на тестовой выборке результат получился немного хуже, но совсем не критично!

Проверим нашу модель на актуальность через auc_roc.

In [44]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
auc_roc

0.7758625850014366

Наша модель в сравнении со случайными показателями показывает более высокий результат.Признаем ее адекватной!)


Подведем итог.

На основе данных клиентов банка было построено несколько моделей для попытки спрогнозировать риск ухода клиента.
В основу исследований легли 3 модели : LogisticRegression, DecisionTreeClassifier и RandomForestClassifier.
И для каждой из них применено 2 метода борьбы с дисбалансами! это взмешимание классов и увеличение выборки.
так же к каждой из получившихся моделей примена метрика AUC-ROC. 

Самой подходящей моделью для анализа риска потерять клиента получилась 'RandomForestClassifier' с взвешиванием классов, максимальным количеством - 23 деревьев и максимальной глубиной - 7!  Эта модель так же успешно прошла тестирование, превысив заданный порог в 0.59 и проверку на адекватность с показателем AUC-ROC в 0.77